# Setup Notebook

In [1]:
# setup autoreload (doesn't work in colab)
# %load_ext autoreload
# %autoreload 2

In [5]:
# import standard libraries
import numpy as np

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, ConcatDataset

import matplotlib.pyplot as plt

In [6]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

Using device cpu


In [4]:
# install weight and biases library
!pip install wandb
import wandb

ERROR: Operation cancelled by user


In [1]:
# setup repo
!rm -r dl-computer-vision
!git clone https://github.com/simonamtoft/dl-computer-vision.git

import sys
sys.path.append('dl-computer-vision')

import models as m
import helpers as h
import data as d

Cloning into 'dl-computer-vision'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 99 (delta 40), reused 79 (delta 26), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [6]:
# Define name of project on weight and biases site
project_name = "project-1-2"

# Project 1.2: Object Detection

In [7]:
# login to weight and biases
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
inc = 3     # input channels
ouc = 11    # output channels (classes)
config = {
    'epochs': 20,
    'batch_size': 64,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'conv_dim': [
        [inc, None],
        [16, 5],
        [16, 5], 
        [16, 3],
        [16, 3],
        [32, 3],
        [32, 1],
        [ouc, 1]
    ],
    'maxpool_idx': [2, 3],
    'batch_norm': False,
    'step_lr': [True, 1, 0.8],
}

## Load Data

In [11]:
# Define data transforms
svhn_train_transform = transforms.Compose([
  transforms.ToTensor()
])
svhn_test_transform = transforms.Compose([
  transforms.ToTensor()
])

In [17]:
# Download datasets 
svhn_trainset = datasets.SVHN(
    "./data", 
    split="train", 
    transform=svhn_train_transform, 
    download=True
)
cifar_trainset = datasets.CIFAR10(
    "./data", 
    train=True, 
    transform=transforms.ToTensor(), 
    target_transform=lambda x: 10, 
    download=True
)
svhn_testset = datasets.SVHN(
    "./data", 
    split="test", 
    transform=svhn_test_transform, 
    download=True
)
cifar_testset = datasets.CIFAR10(
    "./data", 
    train=False, 
    transform=transforms.ToTensor(), 
    target_transform=lambda x: 10, 
    download=True
)

Using downloaded and verified file: ./data/train_32x32.mat
Files already downloaded and verified
Using downloaded and verified file: ./data/test_32x32.mat
Files already downloaded and verified


In [18]:
# Concat datasets and setup loaders
trainset = ConcatDataset([svhn_trainset, cifar_trainset])
testset = ConcatDataset([svhn_testset, cifar_testset])

train_loader = DataLoader(
    trainset, batch_size=config["batch_size"], shuffle=True, num_workers=2
)
test_loader = DataLoader(
    testset, batch_size=config["batch_size"], shuffle=False, num_workers=2
)

## Train Model

In [9]:
# Instantiate model
model = m.PureCNN(config).to(device)
print(model)

PureCNN(
  (conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU()
    (5): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(32, 11, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
  )
)


In [25]:
print(model)

StandardCNN(
  (conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), paddin